# Extension Substep 1: Recipe Step Localization using HiERO

## Objective
Segment individual steps of recipe videos using a zero-shot clustering-based approach (HiERO). 

## Workflow
1. **Load pre-trained HiERO model**: Uses a graph neural network trained on hierarchical temporal modeling
2. **Extract hierarchical features**: Computes multi-scale temporal representations from video features
3. **Cluster and segment**: Uses spectral clustering with median filtering to identify temporal boundaries
4. **Compute step embeddings**: Averages video features within detected step boundaries to obtain step-level representations

## Output
For each video: 
- A list of tuples `(start_time, end_time)` defining step boundaries
- A sequence of step-level embeddings (averaged features for each detected step)
- Cluster labels indicating temporal boundaries

## Key Technologies
- **HiERO**: Hierarchical temporal segmentation using graph neural networks
- **Spectral Clustering**: Zero-shot clustering for temporal boundary detection
- **Median Filtering**: Temporal smoothing to reduce noise in segmentation labels

In [ ]:
# @title 1. Setup Environment & Path Configuration
"""
Initialize the environment by mounting Google Drive and installing required dependencies.
This setup ensures all necessary libraries and project code are accessible.
"""
import os
import sys
from google.colab import drive

# Mount Google Drive to access project files and data
drive.mount('/content/drive')

# --- PATH CONFIGURATION ---
# Define the project root directory where files are stored on Drive
DRIVE_ROOT = "/content/drive/MyDrive/AML_Project/Extension/step1_HiERO"

# Locate the HiERO code directory
REPO_DIR = os.path.join(DRIVE_ROOT, "HiERO")

# Verify the directory exists
if not os.path.exists(REPO_DIR):
    raise FileNotFoundError(f"Could not find code directory at: {REPO_DIR}. Please ensure the 'HiERO' folder has been correctly moved.")

print(f"Working directory set to: {REPO_DIR}")

# Install Dependencies
# Install required packages from requirements.txt with PyTorch GPU support
%cd "{REPO_DIR}"
!pip install -r requirements.txt -f https://data.pyg.org/whl/torch-2.4.0+cu124.html --extra-index-url https://download.pytorch.org/whl/
!pip install hydra-core --upgrade

# Add HiERO to Python path
# Essential for Python to locate modules such as 'models', 'utils', etc.
if REPO_DIR not in sys.path:
    sys.path.append(REPO_DIR)

print("Environment setup completed successfully.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Working directory set to: /content/drive/MyDrive/AML_Project/Extension/step1_HiERO/HiERO
/content/drive/.shortcut-targets-by-id/1vxgD6uYnr2LpQalDA9eOLzCE7O5v_LHt/AML_Project/Extension/step1_HiERO/HiERO
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/
Looking in links: https://data.pyg.org/whl/torch-2.4.0+cu124.html


In [ ]:
# @title 2. Utility Functions (Model & Clustering)
"""
Core utility functions for HiERO model inference, hierarchical feature extraction,
and temporal segmentation via spectral clustering.
"""
import torch
import hydra
import numpy as np
from torch_geometric.data import Data
from sklearn.cluster import SpectralClustering
from torch.nn import functional as F

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def build_hiero_model(ckpt_path, input_size=256, depth=2):
    """
    Load the HiERO model from a checkpoint file.
    
    The model is instantiated with configuration saved in the checkpoint,
    enabling hierarchical graph neural network processing.
    
    Args:
        ckpt_path (str): Path to the model checkpoint file
        input_size (int): Input feature dimension (default: 256 for EgoVLP)
        depth (int): Hierarchical depth for feature extraction (default: 2)
    
    Returns:
        torch.nn.Module: Loaded HiERO model in evaluation mode on the appropriate device
    
    Raises:
        FileNotFoundError: If the checkpoint file does not exist
    """
    if not os.path.exists(ckpt_path):
        raise FileNotFoundError(f"Checkpoint not found at: {ckpt_path}")

    print(f"Loading model from {ckpt_path}...")
    weights = torch.load(ckpt_path, map_location=DEVICE)

    # Instantiate model using saved configuration
    model = hydra.utils.instantiate(
        weights["config"]["model"],
        clustering_at_inference=True,
        input_size=input_size,
        _recursive_=False
    ).to(DEVICE)

    model.load_state_dict(weights["model"], strict=False)
    model.eval()
    return model

def extract_hiero_features(model, features, depth=2):
    """
    Perform inference to obtain hierarchical feature representations.
    
    Constructs a temporal graph from input features and extracts
    multi-scale temporal representations at the specified hierarchical depth.
    
    Args:
        model: HiERO model instance
        features (torch.Tensor): Input video features of shape (T, feature_dim)
        depth (int): Hierarchical level for feature extraction (default: 2)
    
    Returns:
        torch.Tensor: Hierarchical features at the specified depth
    """
    features = features.to(DEVICE)

    # Build dummy temporal graph structure
    pos = torch.arange(0, features.shape[0], device=DEVICE).float()
    indices = torch.arange(0, features.shape[0], device=DEVICE)
    batch = torch.zeros_like(indices, dtype=torch.long)
    mask = torch.ones_like(indices, dtype=torch.bool)

    data = Data(x=features.unsqueeze(1), pos=pos, indices=indices, batch=batch, mask=mask)

    with torch.no_grad():
        graphs = model(data)
        out_features = graphs.x[graphs.depth == depth]

    return out_features

def clusterize_segments(features, n_clusters=5, temp=0.05):
    """
    Cluster features using spectral clustering to identify temporal segments.
    
    Computes similarity affinity matrix using cosine distance with temperature scaling,
    then applies spectral clustering to group temporally similar features.
    
    Args:
        features (torch.Tensor): Features to cluster of shape (N, feature_dim)
        n_clusters (int): Target number of clusters (default: 5)
        temp (float): Temperature parameter for affinity computation (default: 0.05)
    
    Returns:
        np.ndarray: Cluster labels for each temporal position
    """
    features = F.normalize(features, p=2, dim=-1)
    affinity = torch.exp((features @ features.T) / temp).cpu().numpy()
    sc = SpectralClustering(n_clusters=n_clusters, affinity="precomputed", assign_labels='kmeans')
    labels = sc.fit_predict(affinity)
    return labels

def get_segments_timestamps(labels, fps, stride=16, depth=2):
    """
    Convert cluster labels to temporal segment boundaries.
    
    Identifies transitions in cluster labels and converts them to
    (start_time, end_time) tuples based on video FPS and feature stride.
    
    Args:
        labels (np.ndarray): Cluster labels for each position
        fps (float): Frames per second of the original video
        stride (int): Frame stride used in feature extraction (default: 16)
        depth (int): Hierarchical depth (affects temporal scale) (default: 2)
    
    Returns:
        list: List of (start_time, end_time) tuples in seconds
    """
    seconds_per_block = (stride / fps) * (2**depth)
    segments = []
    if len(labels) == 0:
        return segments

    current_label = labels[0]
    start_idx = 0

    for i, label in enumerate(labels):
        if label != current_label:
            end_idx = i
            start_time = start_idx * seconds_per_block
            end_time = end_idx * seconds_per_block
            segments.append((start_time, end_time))
            current_label = label
            start_idx = i

    # Append final segment
    start_time = start_idx * seconds_per_block
    end_time = len(labels) * seconds_per_block
    segments.append((start_time, end_time))

    return segments

def average_features_within_steps(original_features, segments, fps, stride=16):
    """
    Compute step-level embeddings by averaging features within temporal boundaries.
    
    For each detected step (segment), pools the original video features that fall
    within the segment boundaries, creating a single representative embedding.
    
    Args:
        original_features (torch.Tensor): Original video features (T, feature_dim)
        segments (list): List of (start_time, end_time) tuples
        fps (float): Frames per second of the video
        stride (int): Frame stride used in feature extraction (default: 16)
    
    Returns:
        np.ndarray: Step-level embeddings, shape (num_segments, feature_dim)
    """
    step_embeddings = []
    feat_duration = stride / fps

    for start, end in segments:
        start_idx = int(start / feat_duration)
        end_idx = int(end / feat_duration)
        start_idx = max(0, start_idx)
        end_idx = min(len(original_features), max(start_idx + 1, end_idx))

        step_feat = original_features[start_idx:end_idx]

        if len(step_feat) > 0:
            avg_feat = torch.mean(step_feat, dim=0)
        else:
            avg_feat = torch.zeros_like(original_features[0])

        step_embeddings.append(avg_feat.cpu().numpy())

    return np.array(step_embeddings)

In [ ]:
# @title 1. Load Metadata & Configure Parameters
"""
Load video metadata from CSV files and build a parameter dictionary
for HiERO processing. Maps each video to the number of recipe steps
and frame rate.
"""
import pandas as pd
import json
import os

# --- PATH CONFIGURATION ---
# Ensure these paths correctly point to your annotations and task graph files
CSV_STEPS_PATH = '/content/drive/MyDrive/AML_Project/annotations-main/annotation_csv/recording_id_step_idx.csv'
CSV_NAMES_PATH = '/content/drive/MyDrive/AML_Project/annotations-main/annotation_csv/average_segment_length.csv'
CSV_TASK_GRAPHS_PATH = '/content/drive/MyDrive/AML_Project/annotations-main/task_graphs'

def load_video_parameters(steps_csv, names_csv):
    """
    Create a dictionary mapping video IDs to their processing parameters.
    
    For each video, determines:
    - n_clusters: The number of unique recipe steps in the task
    - activity_name: The name of the recipe
    - fps: Frame rate of the video (for temporal scaling)
    
    This information is loaded from CSV files and task graph JSON files,
    providing semantic structure for the clustering process.
    
    Args:
        steps_csv (str): Path to CSV file with step indices for each recording
        names_csv (str): Path to CSV file with recipe names and metadata
    
    Returns:
        dict: Mapping from video_id to {n_clusters, activity_name, activity_id, fps}
    
    Raises:
        FileNotFoundError: If CSV files or task graph files cannot be found
    """
    print("Loading metadata...")

    # Load CSV files
    df_steps = pd.read_csv(steps_csv)
    df_names = pd.read_csv(names_csv)

    # Ensure IDs are strings for consistent matching
    df_steps['activity_id'] = df_steps['activity_id'].astype(str)
    df_names['activity_id'] = df_names['activity_id'].astype(str)

    # Merge dataframes on activity_id to associate recipe names with recordings
    df_merged = pd.merge(df_steps, df_names, on='activity_id', how='left')

    video_params = {}

    for _, row in df_merged.iterrows():
        video_id = str(row['recording_id'])

        # --- COMPUTE N_CLUSTERS ---
        # Strategy: Number of clusters equals the number of unique steps in the recipe.
        # Example: step sequence "3,1,4,1,3" -> Unique steps: {1,3,4} -> n_clusters=3
        # HiERO will group semantically similar features; temporal boundaries separate repetitions.
        try:
            step_list = [int(s) for s in str(row['step_indices']).split(',')]
            n_unique_steps = len(set(step_list))
            s = row['activity_name'].replace(' ', '').lower()
            with open(f"{CSV_TASK_GRAPHS_PATH}/{s}.json", "r") as f:
              data = json.load(f)
            num_steps = len(data["steps"])
            # Safety: minimum 2 clusters required for clustering algorithm
            #n_clusters = max(2, n_unique_steps)
        except:
            print(f"Warning: Could not parse steps for {video_id}, using default n_clusters=7")
            num_steps = 7

        # --- FRAME RATE (FPS) ---
        # CSV files do not contain FPS information directly.
        # Default value of 30.0 is used; main processing loop refines this from video metadata.
        fps = 30.0

        video_params[video_id] = {
            'n_clusters': num_steps,
            'activity_name': row['activity_name'],
            'activity_id': row['activity_id'],
            'fps': fps
        }

    print(f"Loaded parameters for {len(video_params)} videos.")
    return video_params

# Execute metadata loading
try:
    VIDEO_PARAMS = load_video_parameters(CSV_STEPS_PATH, CSV_NAMES_PATH)

    # Print sample for verification
    sample_id = list(VIDEO_PARAMS.keys())[0]
    print(f"\nSample video {sample_id}: {VIDEO_PARAMS[sample_id]}")

except Exception as e:
    print(f"Error loading CSV files: {e}")
    # Fallback to empty dictionary to prevent blocking
    VIDEO_PARAMS = {}

Loading metadata...
Loaded parameters for 384 videos.

Esempio 1_7: {'n_clusters': 14, 'activity_name': 'Microwave Egg Sandwich', 'activity_id': '1', 'fps': 30.0}


In [ ]:
# @title HiERO Step Localization (Robust Matching Version + Median Filter)
"""
Main processing pipeline for recipe step localization using HiERO.

Workflow:
1. Load hierarchical features from HiERO model inference
2. Apply spectral clustering to identify temporal clusters
3. Filter noise using median filtering on cluster labels
4. Convert clusters to temporal step boundaries
5. Compute step-level embeddings by averaging original features
6. Save results (segments, embeddings, labels) per video
"""
import os
import glob
import pandas as pd
import numpy as np
import torch
import hydra
from torch_geometric.data import Data
from sklearn.cluster import SpectralClustering
from torch.nn import functional as F
from scipy.ndimage import median_filter  # Temporal smoothing filter

# ==========================================
# 1. CONFIGURATION
# ==========================================

# Input/Output paths
INPUT_DIR_FEAT = '/content/drive/MyDrive/AML_Project/3_EgoVLP/features'
OUTPUT_DIR     = '/content/drive/MyDrive/AML_Project/Extension/step1_HiERO/steps_v4'
PARAMS_CSV     = '/content/drive/MyDrive/AML_Project/Extension/step1_HiERO/video_params_dump.csv'
CKPT_PATH      = '/content/drive/MyDrive/AML_Project/Extension/step1_HiERO/HiERO/checkpoints/hiero_egovlp.pth'

# Fixed HiERO parameters
STRIDE = 16  # Frame stride in feature extraction (affects temporal resolution)
DEPTH = 2    # Hierarchical depth for feature extraction
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# ==========================================
# 2. CORE FUNCTIONS
# ==========================================

def load_params_from_dump(csv_path):
    """
    Load video processing parameters (n_clusters and fps) from CSV dump file.
    
    Args:
        csv_path (str): Path to the parameters CSV file
    
    Returns:
        dict: Mapping from video_id to {n_clusters, fps}
    
    Raises:
        FileNotFoundError: If CSV file does not exist
    """
    print(f"Loading parameters from: {csv_path}")
    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"CSV not found: {csv_path}")

    df = pd.read_csv(csv_path)

    # Ensure video_id is string and whitespace-trimmed
    df['video_id'] = df['video_id'].astype(str).str.strip()

    # Build fast lookup dictionary: id -> {n_clusters, fps}
    params_map = {}
    for _, row in df.iterrows():
        # Safe fps fallback to 30.0 if missing
        fps_val = float(row['fps']) if 'fps' in row and not pd.isna(row['fps']) else 30.0

        params_map[row['video_id']] = {
            'n_clusters': int(row['n_clusters']),
            'fps': fps_val
        }
    return params_map

def find_matching_id(filename, valid_ids):
    """
    Find which valid video ID is contained in the feature filename.
    
    Attempts multiple matching strategies (exact match, prefix match)
    to robustly identify the video ID from filename.
    
    Args:
        filename (str): Feature file name
        valid_ids (set): Set of valid video IDs from parameters
    
    Returns:
        str or None: Matched video ID, or None if no match found
    """
    clean_name = os.path.splitext(filename)[0]
    if clean_name in valid_ids:
        return clean_name

    # Try prefix matching
    candidates = []
    for vid in valid_ids:
        if filename.startswith(vid + "_") or filename.startswith(vid + "."):
            candidates.append(vid)

    if candidates:
        return max(candidates, key=len)

    return None

def build_hiero_model(ckpt_path):
    """
    Load HiERO model from checkpoint (wrapper for utility function).
    
    Args:
        ckpt_path (str): Path to model checkpoint
    
    Returns:
        torch.nn.Module: Loaded model in evaluation mode
    """
    weights = torch.load(ckpt_path, map_location=DEVICE)
    model = hydra.utils.instantiate(weights["config"]["model"], clustering_at_inference=True, input_size=256, _recursive_=False).to(DEVICE)
    model.load_state_dict(weights["model"], strict=False)
    model.eval()
    return model

def extract_hiero_features(model, features):
    """
    Extract hierarchical features using HiERO model (wrapper for utility function).
    
    Args:
        model: HiERO model instance
        features (torch.Tensor): Input features (T, feature_dim)
    
    Returns:
        torch.Tensor: Hierarchical features at depth=DEPTH
    """
    features = features.to(DEVICE)
    pos = torch.arange(0, features.shape[0], device=DEVICE).float()
    indices = torch.arange(0, features.shape[0], device=DEVICE)
    batch = torch.zeros_like(indices, dtype=torch.long)
    mask = torch.ones_like(indices, dtype=torch.bool)
    data = Data(x=features.unsqueeze(1), pos=pos, indices=indices, batch=batch, mask=mask)
    with torch.no_grad():
        graphs = model(data)
        return graphs.x[graphs.depth == DEPTH]

def cluster_and_segment(features, n_clusters, fps):
    """
    Perform spectral clustering and convert cluster labels to temporal segments.
    
    Applies median filtering to the cluster labels to reduce noise and ensure
    temporal stability of segment boundaries.
    
    Args:
        features (torch.Tensor): Hierarchical features to cluster
        n_clusters (int): Target number of recipe steps
        fps (float): Frame rate for temporal scaling
    
    Returns:
        tuple: (segments, labels) where:
            - segments: List of (start_time, end_time) tuples
            - labels: Filtered cluster labels array
    """
    features_norm = F.normalize(features, p=2, dim=-1)
    affinity = torch.exp((features_norm @ features_norm.T) / 0.05).cpu().numpy()
    assign_labels = 'discretize' if n_clusters < 3 else 'kmeans'

    # 1. Perform spectral clustering
    sc = SpectralClustering(n_clusters=n_clusters, affinity="precomputed", assign_labels=assign_labels, random_state=42)
    labels = sc.fit_predict(affinity)

    # ============================================================
    # MEDIAN FILTERING FOR TEMPORAL SMOOTHING
    # ============================================================
    # Kernel size=5 reduces noise and prevents overly fragmented segments.
    # Ensures temporal stability of cluster labels before computing boundaries.
    if len(labels) > 0:
        labels = median_filter(labels, size=5)
    # ============================================================

    seconds_per_block = (STRIDE / fps) * (2**DEPTH)
    segments = []
    if len(labels) == 0: 
        return segments, labels

    # Identify cluster transitions to define segment boundaries
    current_label = labels[0]
    start_idx = 0
    for i, label in enumerate(labels):
        if label != current_label:
            segments.append((start_idx * seconds_per_block, i * seconds_per_block))
            current_label = label
            start_idx = i
    segments.append((start_idx * seconds_per_block, len(labels) * seconds_per_block))
    return segments, labels

def compute_step_embeddings(original_features, segments, fps):
    """
    Compute step-level embeddings by averaging features within segment boundaries.
    
    Args:
        original_features (torch.Tensor): Original video features (T, feature_dim)
        segments (list): Detected step boundaries as (start_time, end_time) tuples
        fps (float): Frame rate for temporal scaling
    
    Returns:
        np.ndarray: Step-level embeddings (num_steps, feature_dim)
    """
    embeddings = []
    feat_duration = STRIDE / fps
    for start, end in segments:
        start_idx = max(0, int(start / feat_duration))
        end_idx = min(len(original_features), max(start_idx + 1, int(end / feat_duration)))
        step_feat = original_features[start_idx:end_idx]
        avg = torch.mean(step_feat, dim=0) if len(step_feat) > 0 else torch.zeros_like(original_features[0])
        embeddings.append(avg.cpu().numpy())
    return np.array(embeddings)

# ==========================================
# 3. MAIN PROCESSING LOOP
# ==========================================

# Create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Load video parameters from CSV
video_params = load_params_from_dump(PARAMS_CSV)
valid_ids_set = set(video_params.keys())

# Load HiERO model
print("Initializing HiERO model...")
model = build_hiero_model(CKPT_PATH)

# Discover all feature files
feat_files = glob.glob(os.path.join(INPUT_DIR_FEAT, '*.npy')) + glob.glob(os.path.join(INPUT_DIR_FEAT, '*.npz'))
print(f"Found {len(feat_files)} feature files. Processing matched videos...")

processed_count = 0
skipped_count = 0

for fpath in feat_files:
    filename = os.path.basename(fpath)

    # 1. Identify video ID from filename
    video_id = find_matching_id(filename, valid_ids_set)

    if not video_id:
        continue

    n_clusters_target = video_params[video_id]['n_clusters']
    fps = video_params[video_id]['fps']

    try:
        # 2. Load original video features
        if fpath.endswith('.npy'):
            feats_np = np.load(fpath)
        else:
            # Handle .npz files with flexible key discovery
            d = np.load(fpath)
            feats_np = d['features'] if 'features' in d else d[list(d.keys())[0]]
        feats_tensor = torch.from_numpy(feats_np).float()

        # 3. Extract hierarchical features using HiERO
        hiero_feats = extract_hiero_features(model, feats_tensor)

        actual_clusters = min(n_clusters_target, len(hiero_feats))
        if actual_clusters < 2:
            print(f"[SKIP] {video_id}: Too short or insufficient features for clustering.")
            skipped_count += 1
            continue

        # 4. Perform clustering and temporal segmentation (with median filtering)
        segments, labels = cluster_and_segment(hiero_feats, actual_clusters, fps)

        # 5. Compute step-level embeddings by averaging features within segments
        step_embeddings = compute_step_embeddings(feats_tensor, segments, fps)

        # 6. Save results to disk
        save_path = os.path.join(OUTPUT_DIR, f"{video_id}_steps.npz")
        np.savez(save_path, segments=segments, embeddings=step_embeddings, labels=labels)

        print(f"[OK] {video_id} -> Saved (K={actual_clusters}, Segments={len(segments)})")
        processed_count += 1

    except Exception as e:
        print(f"[ERROR] {video_id} ({filename}): {e}")

print(f"\n=== Processing Complete ===")
print(f"Successfully processed: {processed_count} videos")
print(f"Skipped: {skipped_count} videos")
print(f"Output directory: {OUTPUT_DIR}")

Loading parameters from: /content/drive/MyDrive/AML_Project/Extension/step1_HiERO/video_params_dump.csv


/tmp/ipython-input-727532447.py:77: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  weights = torch.load(ckpt_path, map_location=DEVICE)


Found 768 feature files. Processing matched videos...
[OK] 10_16 -> Saved (K=17, Segments=35)
[OK] 10_18 -> Saved (K=17, Segments=34)
[OK] 10_24 -> Saved (K=17, Segments=40)
[OK] 10_26 -> Saved (K=17, Segments=26)
[OK] 10_31 -> Saved (K=17, Segments=23)
[OK] 10_42 -> Saved (K=17, Segments=30)
[OK] 10_46 -> Saved (K=17, Segments=31)
[OK] 10_47 -> Saved (K=17, Segments=25)
[OK] 10_48 -> Saved (K=17, Segments=31)
[OK] 10_50 -> Saved (K=17, Segments=17)
[OK] 10_6 -> Saved (K=17, Segments=16)
[OK] 10_7 -> Saved (K=17, Segments=32)
[OK] 12_10 -> Saved (K=9, Segments=18)
[OK] 12_119 -> Saved (K=9, Segments=15)
[OK] 12_12 -> Saved (K=9, Segments=21)
[OK] 12_13 -> Saved (K=9, Segments=23)
[OK] 12_15 -> Saved (K=9, Segments=19)
[OK] 12_16 -> Saved (K=9, Segments=16)
[OK] 12_17 -> Saved (K=9, Segments=9)
[OK] 12_19 -> Saved (K=9, Segments=19)
[OK] 12_26 -> Saved (K=9, Segments=14)
[OK] 12_2 -> Saved (K=9, Segments=16)
[OK] 12_38 -> Saved (K=9, Segments=16)
[OK] 12_41 -> Saved (K=9, Segments=23)
[